In [ ]:
# First: load the model with vllm:

# vllm serve unsloth/gemma-3-1b-it --port 8008 --max-model-len 2500 --served-model-name unsloth/gemma-3-1b-it --gpu-memory-utilization 0.8 --dtype bfloat16 --api-key xxx

In [1]:
import subprocess
from pathlib import Path
from collections import Counter

In [ ]:
# Example 1: Direct prompting mode. Returns a single unslopped output.

if True:
    cmd = [
        'python3', 'main.py',
        #'--api-base-url', 'http://localhost:8000/v1',
        '--api-key', 'xxx',
        '--model-name', 'Qwen/Qwen3-4B',
        '--chat-template-model-id', 'Qwen/Qwen3-4B',
        '--logging-level', 'INFO',
        '--slop-phrases-file', 'banlists/slop_phrases.json',
        '--top-n-slop-phrases', '500',
        '--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
        '--max-new-tokens', '500',
        '--prompt', 'Write a story about a princess. '
    ]
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=True)

In [2]:
# Example 2: Generate an unslopped dataset using prompts from a huggingface sharegpt dataset

# Make sure you've launched the vllm openai-compatible server first. You can use something like:

# vllm serve unsloth/gemma-3-4b-it --port 8000 --max-model-len 2500 --served-model-name unsloth/gemma-3-4b-it --gpu-memory-utilization 0.95 --dtype bfloat16 --api-key xxx


THREADS  = 40
MAX_PROMPTS = 40

OUT_JSONL     = 'creative_writing_generations.jsonl'
HUMAN_PROFILE = 'data/human_writing_profile.json'


cmd = [
    'python3', 'main.py',
    #'--api-base-url', 'http://localhost:8000/v1',
    '--api-key', 'xxx',
    #'--model-name', 'unsloth/gemma-3-4b-it',
    '--model-name', 'sam-paech/Qwen3-14B-antislop-exp17',
    '--output-jsonl', OUT_JSONL,
    #'--input-hf-dataset', 'Nitral-AI/Reddit-SFW-Writing_Prompts_ShareGPT',
    '--input-hf-dataset', 'Nitral-AI/Reddit-NSFW-Writing_Prompts_ShareGPT',
    #'--refusals-file', 'refusals.jsonl',
    '--hf-dataset-split', 'train',
    '--threads', str(THREADS),
    '--max-prompts', str(MAX_PROMPTS),
    '--logging-level', 'INFO',
    #'--slop-phrases-file', 'banlists/slop_phrases.jsonl',
    #'--top-n-slop-phrases', '500',
    '--top-n-slop-phrases', '0',
    #'--regex-blocklist-file', 'banlists/regex_not_x_but_y.json',
    '--max-new-tokens', '2000',
    '--request-mode', 'chunk',  # chunk / stream (stream mode might be buggy)
    '--chunk-size', '20', # how many tokens we request from the api before checking it for new violations
    '--chat-template-model-id', "sam-paech/Qwen3-14B-antislop-exp17",
    '--ftpo-pairs-jsonl', 'ftpo-pairs.jsonl'
]
print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=True)


Running: python3 main.py --api-key xxx --model-name sam-paech/Qwen3-14B-antislop-exp17 --output-jsonl creative_writing_generations.jsonl --input-hf-dataset Nitral-AI/Reddit-NSFW-Writing_Prompts_ShareGPT --hf-dataset-split train --threads 40 --max-prompts 40 --logging-level INFO --top-n-slop-phrases 0 --max-new-tokens 2000 --request-mode chunk --chunk-size 20 --chat-template-model-id sam-paech/Qwen3-14B-antislop-exp17 --ftpo-pairs-jsonl ftpo-pairs.jsonl
INFO mode: Progress bar and ban events will be printed. Most logs suppressed. Effective script level: INFO
Extracting HF prompts:   1%|          | 39/5938 [00:00<00:01, 4864.48prompt/s]
Preparing to process 40 new prompts in this run.
BANNED (prompt_idx=11, type=ngram, ngram='eyes narrowing')               
BANNED (prompt_idx=30, type=ngram, ngram='looked like')                   
BANNED (prompt_idx=37, type=ngram, ngram='felt like')                    
BANNED (prompt_idx=5, type=ngram, ngram='felt sense')                    
Batch Gener

Token indices sequence length is longer than the specified maximum sequence length for this model (1381 > 512). Running this sequence through the model will result in indexing errors


BANNED (prompt_idx=10, type=ngram, ngram='felt like')                            
BANNED (prompt_idx=36, type=ngram, ngram='first time felt')                      
Batch Generating:  10%|█         | 4/40 [01:45<22:05, 36.83s/prompt, 448.1 tok/s]<|user|>
Writing prompt: In a world where people must have weekly sex to survive, you are an emergency services sex worker.

Write 1000 words to this prompt. Your response:

<|assistant|>
In a world where people must have weekly sex to survive, the human race has been forever changed. The original cause of this new necessity was never quite understood—some claimed it was a virus, others thought it was an experiment gone wrong. Whatever it was, the truth was that every human being now requires sexual activity on a weekly basis in order to stay alive. The effects were immediate and dramatic. The world's population was thrown into chaos. Cities were abandoned, families broke apart, and the psychological and emotional toll on individuals was immense

KeyboardInterrupt: 